In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2,658 kB]
Hit:14 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:15 htt

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show(truncate = False)

+-----------+-----------+--------------+----------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# Count the # of pet_product records in dataset 1
df.count()

2643619

In [5]:
from pyspark.sql.functions import to_date
# Transform DataFrame
review_id_table = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date","yyyy-MM-dd").alias("review_date")])
review_id_table.show(truncate = False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|REAKC26P07MDN |28794885   |B00Q0K9604|510387886     |2015-08-31 |
|R3NU7OMZ4HQIEG|11488901   |B00MBW5O9W|912374672     |2015-08-31 |
|R14QJW3XF8QO1P|43214993   |B0084OHUIO|902215727     |2015-08-31 |
|R2HB7AX0394ZGY|12835065   |B001GS71K2|568880110     |2015-08-31 |
|RGKMPDQGSAHR3 |26334022   |B004ABH1LG|692846826     |2015-08-31 |
|R1DJCVPQGCV66E|22283621   |B00AX0LFM4|590674141     |2015-08-31 |
|R3V52EAWLPBFQG|14469895   |B00DQFZGZ0|688538603     |2015-08-31 |
|R3DKO8J1J28QBI|50896354   |B00DIRF9US|742358789     |2015-08-31 |
|R764DBXGRNECG |18440567   |B00JRCBFUG|869798483     |2015-08-31 |
|RW1853GAT0Z9F |50502362   |B000L3XYZ4|501118658     |2015-08-31 |
|R33GITXNUF1AD4|33930128   |B00BOEXWFG|454737777     |2015-08-31 |
|R1H7AVM81TAYRV|43534290   |B001HBBQKY|420905252     |2015-08-

In [6]:
# This table will contain only unique values
product_df = df.select(['product_id', 'product_title']).drop_duplicates()
product_df.show(truncate = False)



+----------+-------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                      |
+----------+-------------------------------------------------------------------------------------------------------------------+
|B00134HSYS|Special Edition Pet Stroller                                                                                       |
|B00BS6NPBG|High Tech Pet 6-Pack Electronic Collar Battery for Model MS-4 and MS-5                                             |
|B000F930FS|PetSafe ScoopFree Cat Litter Box Privacy Hood, Use with ScoopFree Self-Cleaning Cat Litter Box, 2 Color Options    |
|B000FJ9QTW|Insight ActiviToys Olympia Rings Bird Toys                                                                         |
|B000ALY0OQ|SmartCat Bootsie's Three-in-One Cat Scratcher                                        

In [7]:
#  Customer table for first data set
customer_df = df.groupby('customer_id').count().withColumnRenamed('count', 'customer_count')
customer_df.show(truncate = False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|10270641   |1             |
|18365872   |1             |
|16711087   |1             |
|10742726   |2             |
|41169638   |1             |
|43622307   |1             |
|24540309   |2             |
|28258386   |1             |
|35329257   |2             |
|14552054   |1             |
|14529507   |5             |
|45392827   |5             |
|47282953   |1             |
|8201930    |1             |
|20109760   |2             |
|16405801   |4             |
|15056685   |21            |
|20840575   |2             |
|39048303   |1             |
|5596610    |1             |
+-----------+--------------+
only showing top 20 rows



In [8]:
# vine table
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show(truncate = False)

+--------------+-----------+-------------+-----------+----+
|review_id     |star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|REAKC26P07MDN |5          |0            |0          |N   |
|R3NU7OMZ4HQIEG|2          |0            |1          |N   |
|R14QJW3XF8QO1P|5          |0            |0          |N   |
|R2HB7AX0394ZGY|5          |0            |0          |N   |
|RGKMPDQGSAHR3 |5          |0            |0          |N   |
|R1DJCVPQGCV66E|5          |0            |0          |N   |
|R3V52EAWLPBFQG|3          |0            |0          |N   |
|R3DKO8J1J28QBI|2          |0            |0          |N   |
|R764DBXGRNECG |5          |1            |1          |N   |
|RW1853GAT0Z9F |5          |0            |0          |N   |
|R33GITXNUF1AD4|2          |0            |0          |N   |
|R1H7AVM81TAYRV|1          |2            |2          |N   |
|R2ZOYAQZNNZZWV|5          |0            |0          |N   |
|R2FN1H3CGW6J8H|1          |0           

In [9]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-2.ctzgoawfyx4y.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write DataFrame to active_user table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)